# Stray Dog Management – Data Analytics Project
This notebook demonstrates a full analytics workflow using **Python**, **SQL (SQLite)**, and the provided CSV datasets.

**Data files:** `data/dim_state.csv`, `data/dim_city.csv`, `data/fact_city_month.csv`

**Database:** `sql/stray_dog.sqlite` (pre-loaded)


In [ ]:

import os, sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
SQL_DIR = os.path.join(PROJECT_ROOT, "sql")

dim_state = pd.read_csv(os.path.join(DATA_DIR, "dim_state.csv"))
dim_city = pd.read_csv(os.path.join(DATA_DIR, "dim_city.csv"))
fact = pd.read_csv(os.path.join(DATA_DIR, "fact_city_month.csv"), parse_dates=["date"])

print(dim_state.head(), dim_city.head(), fact.head(), sep="\n\n")


## Quick EDA

In [ ]:

by_state = fact.groupby("state_id")[["bites","vaccinations","sterilizations","complaints"]].sum().reset_index()
by_state = by_state.merge(dim_state, on="state_id", how="left")
by_state = by_state.sort_values("bites", ascending=False)
by_state.head(10)


In [ ]:

features = fact[["bites", "accidents", "complaints", "vaccinations", "sterilizations", "adoptions", "rescues", "pickups", "stray_population_estimate"]]
corr = features.corr(numeric_only=True)
corr


In [ ]:

fig = plt.figure(figsize=(8,6))
plt.imshow(corr, interpolation="nearest")
plt.xticks(range(corr.shape[1]), corr.columns, rotation=45, ha="right")
plt.yticks(range(corr.shape[0]), corr.index)
plt.colorbar()
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()


## Time Series Trends

In [ ]:

up_id = int(dim_state.loc[dim_state["state_name"]=="Uttar Pradesh","state_id"].iloc[0])
up_ts = (fact[fact["state_id"]==up_id]
         .groupby("date")[["bites","vaccinations","sterilizations"]]
         .sum()
         .reset_index())
fig = plt.figure(figsize=(9,4))
plt.plot(up_ts["date"], up_ts["bites"], label="Bites")
plt.plot(up_ts["date"], up_ts["vaccinations"], label="Vaccinations")
plt.plot(up_ts["date"], up_ts["sterilizations"], label="Sterilizations")
plt.legend()
plt.title("Uttar Pradesh: Bites vs Interventions Over Time")
plt.xlabel("Date")
plt.ylabel("Counts")
plt.tight_layout()
plt.show()


## Example SQL (SQLite)

In [ ]:

conn = sqlite3.connect(os.path.join(SQL_DIR, "stray_dog.sqlite"))

q1 = '''
SELECT ds.state_name, strftime('%Y-%m', f.date) as ym,
       SUM(f.bites) as total_bites,
       SUM(f.vaccinations) as total_vaccinations,
       SUM(f.sterilizations) as total_sterilizations
FROM fact_city_month f
JOIN dim_state ds ON ds.state_id = f.state_id
GROUP BY ds.state_name, ym
ORDER BY ds.state_name, ym;
'''
df_q1 = pd.read_sql(q1, conn)
df_q1.head()


In [ ]:

q2 = '''
SELECT dc.city_name, ds.state_name, SUM(f.complaints) AS complaints
FROM fact_city_month f
JOIN dim_city dc ON dc.city_id = f.city_id
JOIN dim_state ds ON ds.state_id = f.state_id
GROUP BY dc.city_name, ds.state_name
ORDER BY complaints DESC
LIMIT 10;
'''
pd.read_sql(q2, conn)


## Save a Cleaned Analytics Extract

In [ ]:

export_cols = ["date","year","month","state_id","city_id","bites","accidents","complaints","vaccinations","sterilizations","adoptions","rescues","pickups","stray_population_estimate","budget_spent_lakhs"]
clean = fact[export_cols].copy()
clean.to_csv(os.path.join(DATA_DIR, "fact_clean_export.csv"), index=False)
clean.head()
